In [1]:
# Core scverse libraries
import scanpy as sc
import anndata as ad

# Data retrieval
import pooch
import polars as pl

from lets_plot import (
    ggplot,
    geom_violin,
    geom_point,
    geom_jitter,
    geom_blank,
    aes,
    theme_classic,
    LetsPlot,
    layer_tooltips,
    gggrid,
    ggtb,
    ggsize,
)

LetsPlot.setup_html()

from typing import Literal

In [ ]:
def violin(data, key, fill="#FF00FF", trim=False, tooltips="none"):
    # check if data is an AnnData object
    if not isinstance(data, sc.AnnData):
        raise ValueError("data must be an AnnData object")
    else:
        frame = pl.from_pandas(data.obs, include_index=True).rename({"None": "ID"})
    # check if key is in the columns
    if key not in frame.columns:
        msg = f"key must be a column in the AnnData object, but {key} is not in the columns"
        raise ValueError(msg)

    vln = (
        ggplot(data=frame)
        + geom_violin(data=frame, mapping=aes(y=key), fill=fill, color="#6f6f6f", trim=trim)
        + geom_jitter(data=frame, mapping=aes(y=key), alpha=0.7, size=0.5, tooltips=layer_tooltips(tooltips))
        + theme_classic()
    )
    return vln

In [ ]:
def violins(data, keys: list, interactive=False, **kwargs):
    plots = list()
    for key in keys:
        plots.append(violin(data, key=key, **kwargs))

    side2side = gggrid(plots)

    if interactive:
        return side2side + ggtb()
    else:
        return side2side

In [5]:
adata = sc.read("pbmc3k_pped.h5ad")